# Получаем данные из Amocrm по API

1. Подключаемся впервые
2. Обновляем токен доступа
3. Получаем данные
4. Готовим данные к анализу в Pandas

In [1]:
import requests
import json
import os

from urllib.parse import urlparse, parse_qs, quote, urlencode
from dotenv import load_dotenv

## Часть 1. Подключаемся в первый раз

Начнем с того, что для получения доступа к API, вам нужно иметь агентский доступ к аккаунту. Если доступ есть, идите в настройки кабинета, в раздел API и создавайте новую интеграцию.

Инструкция: https://www.amocrm.ru/developers/content/oauth/easy-auth

Вам надо получить 3 токена:
1. integration id
2. secret key
3. authentication code

Первые два сохраните в окружении. Если не знаете как, сделайте паузу и погуглите `python dotenv`. Authentication code вам понадобится только один раз, его можно запихнуть в переменную.

In [ ]:
# Загружаем в переменные полученные токены
dotenv_path = '.env'
load_dotenv(dotenv_path)

amo_secret_key = os.environ.get("amo_sercret_key")
amo_integration_id = os.environ.get("amo_integration_id")

amo_auth_code='вставьте сюда ваш authentication code'

In [57]:
subdomain = '...' # вставьте название вашего поддомента

base_url = 'https://{}.amocrm.ru'.format(subdomain)

In [6]:
def get_tokens(base_url, client_id, clien_secret, auth_code):
    """Gets auth tokens for Amocrm"""
        
    endpoint = '/oauth2/access_token'
    method = 'POST'

    headers = {
        'User-Agent': 'amoCRM-oAuth-client/1.0'
    }
 
    params = {
        'client_id': client_id,
        'client_secret': clien_secret,
        'grant_type': 'authorization_code',
        'code': auth_code,
        'redirect_uri': 'https://smysl.io'
    }
    res = requests.post(url=base_url+endpoint, data=params, headers=headers)
    return json.loads(res.text)

In [ ]:
def write_to_env(refresh_token, var_name='amo_refresh_token', dotenv_path='.env'):
    """Writes to a .env file"""
    
    with open(dotenv_path, 'r') as f:
        lines = f.readlines()
        d = {k: v for k, v in [l.replace('\n','').split('=') for l in lines]}    
        d[var_name] = refresh_token
    to_write = [k + '=' + v for k, v in d.items()]
    
    with open(dotenv_path, 'w') as f:
        for l in to_write:
            f.write(l + '\n')

In [7]:
# получаем токен впервые
creds = get_tokens(base_url, amo_integration_id, amo_secret_key, amo_auth_code)

# сразу же сохраняем refresh_token в окружении - он нам понадобится
write_to_env(creds['refresh_token'])

## Часть 2. Освежаем access_token

В конце прошлой части мы получили access_token и refresh_token (они записаны в переменной creds). В принципе, уже тогда можно было начинать работать с API. Но есть проблема — access_token будет работать всего 24 часа. Потом его надо обновить.

Для этого в Amocrm есть refresh_token и специальный запрос к API. Отправлем refresh_token, а взамен получаем новый access_token и новый refresh_token. Старый можно выбросить, он больше не будет работать. 

Код из первой части вам нужен только один раз, чтобы получить токен впервые, то этот код надо запускать каждый раз, когда садитесь работать.

In [100]:
# Загружаем токен и настраиваем запрос
amo_refresh_token = os.environ.get("amo_refresh_token")

subdomain = '...' # вставьте название вашего поддомента
base_url = 'https://{}.amocrm.ru'.format(subdomain)

In [91]:
def refresh_tokens(base_url, client_id, clien_secret, refresh_token):
    """Refresh access and refresh tokens in Amocrm"""
        
    endpoint = '/oauth2/access_token'
    method = 'POST'

    headers = {
        'User-Agent': 'amoCRM-oAuth-client/1.0'
    }
 
    params = {
        'client_id': client_id,
        'client_secret': clien_secret,
        'grant_type': 'refresh_token',
        'refresh_token': refresh_token,
        'redirect_uri': 'https://smysl.io'
    }
    res = requests.post(url=base_url+endpoint, data=params, headers=headers)
    return json.loads(res.text)

In [95]:
# освежаем токены и сразу записываем новый refresh_token обратно в .env
creds = refresh_tokens(base_url, amo_integration_id, amo_secret_key, amo_refresh_token)
write_to_env(creds['refresh_token'])

## Часть 3. Работаем с API

Все, доступ есть, теперь, наконец, можно начать работать. Ниже - несколько примеров запросов к API. 

Подробнее - в инструкции https://www.amocrm.ru/developers/content/api/account

In [96]:
access_token = creds['access_token']
refresh_token = creds['refresh_token']

In [97]:
# Получаем информацию об аккаунте

endpoint = '/api/v2/account'
method = 'GET'

headers = {
    'User-Agent': 'amoCRM-oAuth-client/1.0',
    'Authorization': 'Bearer {}'.format(access_token)
}

res = requests.get(url=base_url+endpoint, headers=headers)

In [60]:
# Получаем информацию о сделках

clean_leads = []
ids = set()
i = 0

endpoint = '/api/v2/leads'
method = 'GET'

headers = {
    'User-Agent': 'amoCRM-oAuth-client/1.0',
    'Authorization': 'Bearer {}'.format(access_token)
}

In [ ]:
# Вот так, например, можно выгрузить информацию о всех сделках. 
# Это не самый элегантный код, в продакшн его лучше не выкатывать,
# но со своей задачей он справляется.

while True:
    params = {
        'limit_rows': 500,
        'limit_offset': i
    }

    res = requests.get(url=base_url+endpoint, params=params, headers=headers)
    try:
        new_leads = json.loads(res.text)
        for lead in new_leads['_embedded']['items']:
            if lead['id'] not in ids:
                ids.add(lead['id'])
                clean_leads.append(lead)
        i += 500
    except:
        print('Done')
        break


## Часть 4. Превращаем полученные данные в таблицы и готовим их к анализу

API Amocrm возвращает нам наборы json-ов. Это не очень удобно для анализа. Другое дело таблицы!

Покажу, как можно информацию о сделках перевести в датафрейм.

In [26]:
import pandas as pd
from datetime import timedelta, date

In [83]:
# В сделке есть куча полей, которые могут быть ненужны для анализа. 
# А некоторые полезные поля, типа utm-меток, запрятаны в 5-этажные вложенные словари. 
# Этот код удаляет ненужные поля и достает наружу нужные.

def extract_utm(list_of_tags, tag_name):
    """Extracts a value of the utm-tag of a AMOCrm deal"""
    try:
        tag_value = [x for x in list_of_tags if x['name'] == tag_name][0]['values'][0]['value']
    except:
        tag_value = None
    return tag_value
data = []

for l in clean_leads:
    data.append({
        'id': l['id'],
        'sale': l['sale'],
        'name': l['name'],
        'created_at': l['created_at'],
        'closed_at':  l['closed_at'],
        'pipeline_id': l['pipeline_id'],
        'status_id':   l['status_id'],
        'is_deleted':  l['is_deleted'],
        'utm_source':   extract_utm(l['custom_fields'], 'utm_source'),
        'utm_medium':   extract_utm(l['custom_fields'], 'utm_campaign'),
        'utm_campaign': extract_utm(l['custom_fields'], 'utm_medium'),
        'loss_reason_id': l['loss_reason_id'],
    }
)


In [101]:
# Ну и последнй шаг самый простой — просто конвертируем список словарей в датафрейм.

df = pd.DataFrame(data)
df.set_index('id', inplace=True)

На этом все. Анализируйте на здоровье.

P.S. Спасибо [сообществу Digital God](https://digitalgod.be/) за помощь в подготовке.